In [1]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertJapaneseTokenizer, BertModel
from torch import cuda
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import logging

c:\Users\nukan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
batch_size = 1
max_len = 512

In [31]:
df = pd.read_csv("livedoor_text.csv")
df.head()

,text,category
0,27日に生放送された日本テレビ「バンクーバー2010」には、女子フィギュアスケートで銀メダル...,7
1,「腐女子」という言葉をご存知でしょうか。\nいわゆる漫画やアニメキャラなどの男性同士の恋愛（...,0
2,展示会イベント恒例のおねいさん写真のコーナーでございます \n\n国内最大級の携帯電話や無線...,6
3,芸能界を引退した島田紳助さんが、今月２８日に公開される映画「犬の首輪とコロッケと」に声だけ出...,2
4,お花に包まれた洋館で、イケメン執事に囲まれながら、ゆったりと過ごす午後のひととき……。女の子...,5


In [37]:
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", output_attentions=False, output_hidden_states=False)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
class CreateDataset(Dataset):
  def __init__(self, X, tokenizer, max_len):
    self.X = X
    # self.y = y
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return 1

  def encode(self, tokenizer, text):
      inputs = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation = True
      )
      return inputs

  def __getitem__(self, index):
    text = self.X[index]
    # label = self.y[index]
    ids = []
    mask = []
    inputs = self.encode(tokenizer=self.tokenizer, text=text)
    ids.append(torch.LongTensor(inputs['input_ids']))
    mask.append(torch.LongTensor(inputs['attention_mask']))

    return {
      'ids': ids,
      'mask': mask,
      # 'label': label,
      'text':text,
      # 'userID':userID
    }

In [76]:
# X = df["text"].values[[0]]
# X

text = '27日に生放送された日本テレビ「バンクーバー2010」には、女子フィギュアスケートで銀メダルを獲得した浅田真央が出演した。\n\nメダルを獲得しながらも、自身の演技に満足できず悔し涙を流した運命の日から一夜明け、プレッシャーから解放された安堵感からか、いつもの笑顔がみられるようになった浅田。五輪史上初となる3度のトリプルアクセル（3回転半）を成功させたことには、「今シーズンから3回飛ぶって決めてましたし、それをやりたいっていう思いがあったのでオリンピックで挑戦というよりは、やってきたことを出したいという思いが強かったです」と語った。\n\nまた、五輪という舞台については、「予想していたよりも、すごい大きな舞台だったんだなって、終わってから改めて感じました」と振り返る浅田は、同番組が生放送されることで出演前から緊張していた点に話が及ぶと、「生なんだと。流れてるんだと、日本で・・・」と苦笑いを浮かべた。'
X = np.array([text])

In [77]:
dataset_test = CreateDataset(X, tokenizer, max_len=max_len)

In [78]:
dataset_test[0]

{'ids': [tensor([    2,   971,    32,     7,   128,   333,    26,    20,    10,  5038,
             36, 14353,   785,    38,     7,     9,     6,  1568,  6710,  5461,
             12,  8828,    11,   906,    15,    10,  2752, 28675,   841, 29210,
             14,   793,    15,    10,     8,  6976,    11,   906,    15,   895,
             28,     6,   901,     5,  5913,     7,  9109,   203,   255, 16977,
          28454, 30557,    11,  8272,    10,  9585,     5,    32,    40, 19239,
           9073,     6, 25204,    40,  3766,    26,    20,    10, 18984,   832,
             40,    29,     6,  9749,     5, 18802,    14,   546,   342,   124,
              7,    58,    10,  2752, 28675,     8,  8111,  2737,   176,    13,
            139,    48,   559,     5, 18052, 20614,    23,    48,  3232,   818,
             24,    11,  1320,    26,   191,    10,    45,     7,     9,     6,
             36,   744,   712,    40,    48,   198, 14856,  6172,  2372,    16,
           3913,    10,    15,   

In [68]:
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, pin_memory=True)

In [69]:
# tmp = next(iter(dataloader_test))
# tmp

In [70]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output

In [71]:
# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cpu


In [72]:
# モデル読み込み
net_trained = BertForLivedoor()

save_path = "./single_bert_fine_tuning_livedoor.pth"
net_trained.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [83]:
# ミニバッチの用意
batch = next(iter(dataloader_test))

# GPUが使えるならGPUにデータを送る
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")
inputs = batch["ids"][0].to(device)  # 文章

outputs = net_trained(inputs)
_, pred = torch.max(outputs, 1)  # ラベルを予測

print(f"pred: {pred}")
print(f"pred: {pred.to('cpu').detach().numpy()[0]}")

device: cpu
pred: tensor([7])
pred: 7


In [74]:
outputs

tensor([[-1.8249, -1.1255, -1.5014, -1.5047, -1.0651, -0.1506, -2.5921,  7.0584,
          2.5893]], grad_fn=<AddmmBackward0>)